In [50]:
import os 
import glob
from scipy.io import loadmat, savemat
import pickle as pkl

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from scipy.signal import medfilt

from scipy.signal import resample
from scipy.interpolate import interp1d

In [71]:
train_pkls = glob.glob('../../../datasets/Data_27012021_noisy/train/*.pkl')
print ('Train:', len(train_pkls))
val_pkls = glob.glob('../../../datasets/Data_27012021_noisy/val/*.pkl')
print ('Val:', len(val_pkls))
benchmark_pkls = glob.glob('../../../datasets/Data_27012021_noisy/benchmark/*.pkl')
print ('Benchmark:', len(benchmark_pkls))
raw_mats = glob.glob('../../../datasets/Data_27012021_noisy/raw/*.mat')
print ('Raw:', len(raw_mats))

Train: 100
Val: 100
Benchmark: 4
Raw: 10


In [72]:
fin = open(train_pkls[0], 'rb')
train_sample = pkl.load(fin)
fin.close()

fin = open(val_pkls[0], 'rb')
val_sample = pkl.load(fin)
fin.close()

fin = open(benchmark_pkls[0], 'rb')
benchmark_sample = pkl.load(fin)
fin.close()

raw_sample = loadmat(raw_mats[0])

In [73]:
print ('Train Sample Keys')
print (train_sample.keys())
print (train_sample['speed'].min(), train_sample['torque'].min())
print (train_sample['speed'].max(), train_sample['torque'].max())

print ('\n\nVal Sample Keys')
print (val_sample.keys())
print (val_sample['speed'].min(), val_sample['torque'].min())
print (val_sample['speed'].max(), val_sample['torque'].max())

print ('\n\nBenchmark Sample Keys')
print (benchmark_sample.keys())
print (benchmark_sample['speed'].min(), benchmark_sample['torque'].min())
print (benchmark_sample['speed'].max(), benchmark_sample['torque'].max())


print ('\n\nRaw Sample Keys')
print (raw_sample.keys())
print (raw_sample['Speed_Hz'].min(), raw_sample['Torque_PercTnom'].min())
print (raw_sample['Speed_Hz'].max(), raw_sample['Torque_PercTnom'].max())

Train Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-46.99934659995759 -108.65000728620117
36.30022282079326 223.68110738919583


Val Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'reference_speed', 'torque_time', 'speed_time', 'noisy_current_d', 'noisy_current_q', 'noisy_voltage_d', 'noisy_voltage_q'])
-65.07439460163381 -221.33321791127574
64.59089946035814 81.73566589923782


Benchmark Sample Keys
dict_keys(['voltage_d', 'voltage_q', 'current_d', 'current_q', 'torque', 'speed', 'statorPuls', 'time', 'reference_torque_interp', 'reference_speed_interp', 'reference_torque', 'referen

In [ ]:
def make_mat(pkl_path):
    fin = open(pkl_path, 'rb')
    data = pkl.load(fin)
    fin.close()
    savemat(pkl_path.replace('pkl', 'mat'), data)
    

for pkl_path in train_pkls + val_pkls + benchmark_pkls:
    make_mat(pkl_path)

In [51]:
def ResampleLinear1D(original, targetLen):
    original = np.array(original, dtype=np.float)
    index_arr = np.linspace(0, len(original)-1, num=targetLen, dtype=np.float)
    index_floor = np.array(index_arr, dtype=np.int) #Round down
    index_ceil = index_floor + 1
    index_rem = index_arr - index_floor #Remain

    val1 = original[index_floor]
    val2 = original[index_ceil % len(original)]
    interp = val1 * (1.0-index_rem) + val2 * index_rem
    assert(len(interp) == targetLen)
    return interp

In [69]:
def process_raw_data(mat_path):
    print (mat_path)
    data = loadmat(mat_path)
    
    time = data['Time_VSpdTq'][:, 0]
    speed = data['Speed_Hz'][:, 0]
    torque = data['Torque_PercTnom'][:, 0]
    voltage_d = data['VdRef_V'][: ,0]
    voltage_q = data['VqRef_V'][:, 0]

    current_time = data['Time_I'][:, 0]
    current_d = data['IdMeas_A'][:, 0]
    current_q = data['IqMeas_A'][:, 0]

    
    speed = medfilt(speed, kernel_size=11)
    torque = medfilt(torque, kernel_size=11) 
    voltage_d = medfilt(voltage_d, kernel_size=11)
    voltage_q = medfilt(voltage_q, kernel_size=11)
    current_d = medfilt(current_d, kernel_size=11)
    current_q = medfilt(current_q, kernel_size=11)
    
    f = interp1d(np.arange(current_time.shape[0]), current_d, 'linear')
    
    current_d_down = ResampleLinear1D(current_d, time.shape[0])
    current_q_down = ResampleLinear1D(current_q, time.shape[0])
    
    print ('Time', time.shape, time[1] - time[0])
    print ('Voltage d', voltage_d.shape, voltage_d.min(), voltage_d.max())
    print ('Votlage q', voltage_q.shape, voltage_q.min(), voltage_q.max())
    print ('Speed', speed.shape, speed.min(), speed.max())
    print ('Torque', torque.shape, torque.min(), torque.max())
    print ('Current d', current_d_down.shape, current_d_down.min(), current_d_down.max())
    print ('Current q', current_q_down.shape, current_q_down.min(), current_q_down.max())
    
    data = {'time': time, 
            'noisy_current_d': current_d_down, 
            'noisy_current_q': current_q_down, 
            'noisy_voltage_d': voltage_d, 
            'noisy_voltage_q': voltage_q,
            'noisy_speed': speed,
            'noisy_torque': torque
            }
    
    savemat(mat_path.replace('raw', 'real'), data)
    
for mat_path in raw_mats:
    process_raw_data(mat_path)

../../../datasets/Data_23042020_noisy/raw/OFVC4kW_NoLM_SpeedVariations2_20190419_Data.mat
Time (40928,) 0.004
Voltage d (40928,) -165.09869384765625 162.82205200195312
Votlage q (40928,) -443.0738525390625 442.6241455078125
Speed (40928,) -100.12054443359375 105.560302734375
Torque (40928,) -152.9541015625 155.078125
Current d (40928,) -3.8828125 10.77576200185522
Current q (40928,) -20.104227580034404 21.009605636445777
../../../datasets/Data_23042020_noisy/raw/OFVC4kW_NoLM_paliers_0to100Hz_Data.mat
Time (10224,) 0.004
Voltage d (10224,) -11.551849365234375 43.2843017578125
Votlage q (10224,) -0.16864013671875 324.07012939453125
Speed (10224,) 0.0 100.1434326171875
Torque (10224,) -1.708984375 21.97265625
Current d (10224,) 2.0505130248772296 8.848844725034084
Current q (10224,) -0.4141114809666653 2.5047607421875
../../../datasets/Data_23042020_noisy/raw/OFVC4kW_LM20Hz_TorqueSteps_20190419_Data.mat
Time (51175,) 0.004
Voltage d (51175,) -70.18240356445312 30.327117919921875
Votlage q